# Demonstrating Calibrated Score Generation

This notebook demonstrates the use of the `score_generation` module to create two populations of scores (for 'good' and 'bad' outcomes) that are separated by a specific, targeted AUC value.

This is a key part of the simulation engine, as it allows us to translate an abstract model performance metric (AUC) into concrete, simulated score distributions.

The process is:
1. **Create a Base Distribution**: We start by creating a base 'shape' for our scores using the `BetaMixtureFitter`.
2. **Find the Calibration Factor (Gamma)**: We set a `target_auc` and use the `find_auc_calibration_factor` function to find the `gamma` value needed to achieve it.
3. **Generate Calibrated Scores**: We use this `gamma` to generate two sets of scores with the desired separation.
4. **Verify and Visualize**: We confirm that the resulting AUC is close to our target and plot the distributions to see the separation visually.

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.metrics import roc_auc_score

# Add the project root to the Python path
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from irbstudio.simulation.distribution import BetaMixtureFitter
from irbstudio.simulation.score_generation import (
    find_auc_calibration_factor,
    generate_calibrated_scores
)

print("Imports successful.")

Imports successful.


### 1. Create a Base Score Distribution

First, we need a representative distribution of scores. We'll generate some synthetic data and fit a `BetaMixtureFitter` to it. This fitted distribution will serve as the 'raw material' for our score generation.

In [2]:
np.random.seed(42)
# A simple unimodal distribution is sufficient for this demo
base_data = np.random.beta(a=5, b=20, size=5000)

# Fit the distribution model
base_distribution = BetaMixtureFitter(n_components=1).fit(base_data)

print("Base distribution has been fitted.")

2025-09-09 15:06:30 - irbstudio.simulation.distribution - INFO - Converged after 2 iterations.
Base distribution has been fitted.
Base distribution has been fitted.


### 2. Find the Calibration Factor for a Target AUC

Let's say we want our simulated model to have an AUC of **0.85**. We use the `find_auc_calibration_factor` function to find the `gamma` that will transform our base scores to achieve this.

In [3]:
TARGET_AUC = 0.85

gamma = find_auc_calibration_factor(
    base_distribution=base_distribution, 
    target_auc=TARGET_AUC,
    n_samples_per_dist=20000 # Use a large number for stable optimization
)

print(f"Target AUC: {TARGET_AUC}")
print(f"Found calibrated gamma: {gamma:.4f}")

2025-09-09 15:06:48 - irbstudio.simulation.score_generation - INFO - Searching for gamma to achieve target AUC of 0.85...
2025-09-09 15:06:48 - irbstudio.simulation.score_generation - INFO - Found calibrated gamma of 1.2000.
Target AUC: 0.85
Found calibrated gamma: 1.2000
2025-09-09 15:06:48 - irbstudio.simulation.score_generation - INFO - Found calibrated gamma of 1.2000.
Target AUC: 0.85
Found calibrated gamma: 1.2000


### 3. Generate Calibrated Scores

Now we use this `gamma` to generate our two score populations. We'll create 10,000 scores for the 'good' population (non-defaulters) and 1,000 for the 'bad' population (defaulters).

In [6]:
N_GOOD = 10000
N_BAD = 1000

scores_good, scores_bad = generate_calibrated_scores(
    base_distribution=base_distribution,
    gamma=gamma,
    n_good=N_GOOD,
    n_bad=N_BAD
)

print(f"Generated {len(scores_good)} 'good' scores and {len(scores_bad)} 'bad' scores.")

Generated 10000 'good' scores and 1000 'bad' scores.


### 4. Verify and Visualize the Results

In [7]:
# Verify the AUC
labels = np.concatenate([np.zeros(len(scores_good)), np.ones(len(scores_bad))])
scores = np.concatenate([scores_good, scores_bad])
actual_auc = roc_auc_score(labels, scores)

print(f"Target AUC: {TARGET_AUC}")
print(f"Actual AUC from generated scores: {actual_auc:.4f}")

# Visualize the distributions
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=scores_good,
    name='Good Scores (Non-Defaulters)',
    histnorm='probability density',
    marker_color='blue',
    opacity=0.7
))

fig.add_trace(go.Histogram(
    x=scores_bad,
    name='Bad Scores (Defaulters)',
    histnorm='probability density',
    marker_color='red',
    opacity=0.7
))

fig.update_layout(
    title_text=f'Score Distributions for Target AUC of {TARGET_AUC} (Actual: {actual_auc:.4f})',
    xaxis_title='Score',
    yaxis_title='Density',
    barmode='overlay',
    legend_title='Population'
)

fig.show()

Target AUC: 0.85
Actual AUC from generated scores: 0.8497
